In [23]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [24]:
## 依据已有图片做随机旋转，生成更多样本数据

# 使用ImageDataGenerator处理图片
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

### 数据预处理部分

In [37]:
# 生成更多的样本
def GetMore_Image(cup_type):
        print "Processing {}".format(cup_type)
        
        for i in range(1,11):
            img = load_img('./original_data/cup_{c_type}/{c_type}{num}.png'.format(c_type = cup_type, num = i))
            x = img_to_array(img) 
            x = x.reshape((1,) + x.shape) 
            
            a = 0
            for batch in datagen.flow(x, 
                                      batch_size=1, 
                                      save_to_dir='/home/kelly/project/cup_classification/data/train/{}'.format(cup_type), 
                                      save_prefix='{c_type}_cup'.format(c_type = cup_type), 
                                      save_format='jpeg'):
                a += 1
                if a > 20:
                    break
            
            print "Cup {} of {} is Done!".format(cup_type, i)  


In [38]:
# 存在一个文件夹
for cup_type in ['d','f']:
    GetMore_Image('{}'.format(cup_type))

Processing d
Cup d of 1 is Done!
Cup d of 2 is Done!
Cup d of 3 is Done!
Cup d of 4 is Done!
Cup d of 5 is Done!
Cup d of 6 is Done!
Cup d of 7 is Done!
Cup d of 8 is Done!
Cup d of 9 is Done!
Cup d of 10 is Done!
Processing f
Cup f of 1 is Done!
Cup f of 2 is Done!
Cup f of 3 is Done!
Cup f of 4 is Done!
Cup f of 5 is Done!
Cup f of 6 is Done!
Cup f of 7 is Done!
Cup f of 8 is Done!
Cup f of 9 is Done!
Cup f of 10 is Done!


### 模型训练部分

In [47]:
img_width, img_height = 300, 300

train_data_dir = './data/train/'
validation_data_dir = './data/validation/'
nb_train_samples = 219 * 6
nb_validation_samples = 24
epochs = 50
batch_size = 2

In [48]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [58]:
2 **12

4096

In [77]:
model = Sequential()
model.add(Conv2D(1024, (1, 1), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (1, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (1, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [78]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [79]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [80]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 1314 images belonging to 6 classes.


In [81]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 24 images belonging to 6 classes.


In [121]:
train_generator

In [82]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/50
120/657 [====>.........................] - ETA: 44s - loss: -25.5078 - acc: 0.1375

KeyboardInterrupt: 